# MoI ML CNN Experiment

- Train op fashion mnist
- Laat experimenten zien: kies 2 - 3 params, kies een range en voer tests uit. Doel is 93% accuracy.

In [23]:
import torch
import torch.nn as nn
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../")

In [24]:
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
import gin


In [25]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [26]:
# Load config for this notebook
gin.parse_config_file("cnn.gin")

ParsedConfigFileIncludesAndImports(filename='cnn.gin', imports=[], includes=[])

In [27]:
# Get MNIST data
train_dataloader, test_dataloader = make_dataset.get_MNIST()

In [28]:

# Set some other parameters
import torch.optim as optim
from src.models import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()
from src.models import train_model

## Voer een experiment uit.


### Configuratie voor het experiment

Sla de gewenste structuur voor de lagen op in een lijst configuraties

In [104]:
from dataclasses import dataclass
import itertools 
@dataclass
class NNLayerConfig:
    """Class for storing config and results for a layer."""
    loss: float
    num_params: int
    c1_ksize: int
    c1_stride: int
    c2_ksize: int
    c2_stride: int
    num_filters: int

configs = []  # List of NNLayerConfig objects

# Configuration for the experiment. We ignore padding for now.
# These settings take some tweaking to avoid ending up with runtime
# errors when the sizes end up too small.
kernel_sizes = [1, 2, 3]
filter_nums = [16, 32, 64]
strides = [1, 2]
for config in itertools.product(kernel_sizes, # kernel size in convolution 1
                                strides, # stride size in convolution 1
                                kernel_sizes, # kernel size in convolution 2
                                strides, # stride size in convolution 2
                                filter_nums, # number of filters to use
                                ):
    configs.append(NNLayerConfig(
        loss = 0,
        num_params = 0,
        c1_ksize = config[0],
        c1_stride = config[1],
        c2_ksize = config[2],
        c2_stride = config[3],
        num_filters = config[4],
    ))


Maak een klasse die met een configuratie als input een NN maakt.

In [100]:



# Define model
class ConfigurableNN (nn.Module):

    def __init__(self, config:NNLayerConfig, example_input):
        super().__init__()

        self.config = config
        self.maxpool_ksize = 2 # Hardcoded for now.
        print(self.config)

        # Convolutions are set separately
        self.convolutions = nn.Sequential(
            nn.Conv2d(  1, # First one is always 1
                        self.config.num_filters, 
                        kernel_size = self.config.c1_ksize,
                        stride = self.config.c1_stride,
                        padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=self.maxpool_ksize),
            nn.Conv2d(  self.config.num_filters,
                        self.config.num_filters,
                        kernel_size = self.config.c2_ksize,
                        stride = self.config.c2_stride,
                        padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=self.maxpool_ksize)

        )

        flat_size = self.calculate_flat_size(example_input)
        
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flat_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def calculate_flat_size(self, example_input):
        current = example_input
        for conv in self.convolutions:
            current = conv(current)
        flatten = nn.Flatten()
        return flatten(current).shape[1]

    # Forward is default for now.
    def forward(self, x):
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


Loop door de configuraties heen

In [55]:
def dump_model_steps(model, example_input):
    """
    List the steps of each convolution in a model and show the result for a given input. If possible, also show what happens in a dense layer
    """
    current = example_input
    print("===============")
    print(f'Starting model with shape {current.shape}')
    print("===============")
    if(model.convolutions):
        for conv in model.convolutions:
            print("-----------------")
            print("Convolution:")
            print(conv)
            current = conv(current)
            print(current.shape)
    if(model.dense):
        print("Contents of dense:")
        for dense in model.dense:
            print("-----------------")
            print("Dense:")
            print(dense)
            current = dense(current)
            print(current.shape)


In [105]:
num_epochs = 2 # During development keep very low. 10 is for real tests
learning_rate = 1e-3

# For to be debugging:
x, y = next(iter(train_dataloader))
x.shape, y.shape


for config in configs:
    model = ConfigurableNN(config, x).to(device)
    dump_model_steps(model, x)
    #break
    continue
    model = train_model.trainloop(
        epochs=num_epochs,
        model=model,
        optimizer=optimizer,
        learning_rate=learning_rate,
        loss_fn=loss_fn,
        metrics=[accuracy],
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        log_dir="../models/test/",
        train_steps=len(train_dataloader),
        eval_steps=len(test_dataloader),
    )
    break # During development
    

NNLayerConfig(loss=0, num_params=0, c1_ksize=1, c1_stride=1, c2_ksize=1, c2_stride=1, num_filters=16)
Starting model with shape torch.Size([32, 1, 28, 28])
-----------------
Convolution:
Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1))
torch.Size([32, 16, 28, 28])
-----------------
Convolution:
ReLU()
torch.Size([32, 16, 28, 28])
-----------------
Convolution:
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([32, 16, 14, 14])
-----------------
Convolution:
Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
torch.Size([32, 16, 14, 14])
-----------------
Convolution:
ReLU()
torch.Size([32, 16, 14, 14])
-----------------
Convolution:
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([32, 16, 7, 7])
Contents of dense:
-----------------
Dense:
Flatten(start_dim=1, end_dim=-1)
torch.Size([32, 784])
-----------------
Dense:
Linear(in_features=784, out_features=64, bias=True)
torch.Size([32, 64])
-----------------
Dense:
R

Checkout results using tensorboard: 
- run `tensorboard --logdir models` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`